In [4]:
# Importamos las librerías necesarias

from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Creamos las listas vacías donde vamos a guardar la información obtenida

descriptions=[]
products=[]
prices=[]
original_prices=[] 
ratings=[]
discounts=[]

In [3]:
# Inicializamos un loop que recorra las 45 páginas que tiene la búsqueda de laptops en flipkart 
# para poder extraer la información de todos los productos ofrecidos


for n in range(1,46):
    url= ("https://www.flipkart.com/laptops/pr?sid=6bo%2Cb5g&marketplace=FLIPKART&page=")
    url=url+str(n)
    r = requests.get(url)
    data = r.text
    soup= BeautifulSoup(data, 'html.parser')

# localizamos el id del ancla donde está contenida la información de las laptops
# y a su vez vamos identificando el id del div donde tenemos la información
# de el nombre, precio, rating y la descripcion de cada laptop

    for a in soup.findAll('a',href=True, attrs={'class':'_1fQZEK'}):
        name=a.find('div', attrs={'class':'_4rR01T'})
        price=a.find('div', attrs={'class':'_30jeq3 _1_WHN1'})
        original_price=a.find('div', attrs={'class':'_3I9_wc _27UcVY'})
        discount=a.find('div', attrs={'class':'_3Ay6Sb'})
        rating=a.find('div', attrs={'class':'_3LWZlK'})
        description=a.find('div',attrs={'class':'fMghEO'})

# agregamos a las listas y haremos separación entre los campos que siempre
# tienen información y los que a veces contienen nulos
         
        products.append(name.text)
        prices.append(price.text)
        descriptions.append(description.text)
        try:
            discounts.append(str(discount.span))
        except:
            discounts.append(discount)
        try:
            original_prices.append(original_price.text)
        except:
            original_prices.append(original_price)
        try:
            ratings.append(float(rating.text))
        except:
            ratings.append(None)
        

In [5]:
# Convertimos a  dataframe y lo mostramos

flipk_data = pd.DataFrame({'Product Name':products,'Description':descriptions,'Price rupee':prices, 'Original Price': original_prices,'Discount': discounts,'Rating':ratings})
flipk_data.tail(10)

,Product Name,Description,Price rupee,Original Price,Discount,Rating
974,DELL Ryzen 5 Hexa Core - (16 GB/512 GB SSD/Win...,AMD Ryzen 5 Hexa Core Processor16 GB DDR4 RAMW...,"₹59,990","₹84,879",<span>29% off</span>,NaN
975,DELL Inspiron 5430 Core i7 13th Gen - (16 GB/5...,Intel Core i7 Processor (13th Gen)16 GB LPDDR5...,"₹84,990","₹99,890",<span>14% off</span>,NaN
976,ASUS TUF Gaming F15 with 90WHr Battery Core i5...,Intel Core i5 Processor (11th Gen)16 GB DDR4 R...,"₹69,990","₹89,990",<span>22% off</span>,NaN
977,Infinix Y1 Plus Neo (2023) Celeron Quad Core 1...,Intel Celeron Quad Core Processor (11th Gen)4 ...,"₹20,990","₹29,990",<span>30% off</span>,NaN
978,DELL Core i5 11th Gen - (16 GB/512 GB SSD/Wind...,Intel Core i5 Processor (11th Gen)16 GB DDR4 R...,"₹65,500","₹91,000",<span>28% off</span>,NaN
979,"ASUS Vivobook Pro 15 For Creator, Ryzen 5 Hexa...",AMD Ryzen 5 Hexa Core Processor8 GB DDR4 RAMWi...,"₹57,990","₹76,990",<span>24% off</span>,NaN
980,Infinix ZEROBOOK 13 Intel Core i5 13th Gen - (...,Intel Core i5 Processor (13th Gen)16 GB LPDDR5...,"₹51,990","₹79,990",<span>35% off</span>,NaN
981,HP Chromebook x360 Intel Celeron Dual Core - (...,Intel Celeron Dual Core Processor4 GB LPDDR4 R...,"₹17,990","₹32,295",<span>44% off</span>,4.2
982,Lenovo Ryzen 3 Dual Core 3250U - (4 GB/1 TB HD...,AMD Ryzen 3 Dual Core Processor4 GB DDR4 RAM64...,"₹29,490","₹36,923",<span>20% off</span>,3.6
983,Avita Liber Core i7 8th Gen - (8 GB/256 GB SSD...,Intel Core i7 Processor (8th Gen)8 GB DDR4 RAM...,"₹67,990","₹79,990",<span>15% off</span>,NaN


In [6]:
# Tratamos la columna de Discount para eliminar el tag de span

flipk_data.Discount=flipk_data.Discount.map(lambda x: x[6:9], na_action='ignore')

In [8]:
# Comprobamos el resultado

flipk_data.tail(10)

,Product Name,Description,Price rupee,Original Price,Discount,Rating
974,DELL Ryzen 5 Hexa Core - (16 GB/512 GB SSD/Win...,AMD Ryzen 5 Hexa Core Processor16 GB DDR4 RAMW...,"₹59,990","₹84,879",29%,NaN
975,DELL Inspiron 5430 Core i7 13th Gen - (16 GB/5...,Intel Core i7 Processor (13th Gen)16 GB LPDDR5...,"₹84,990","₹99,890",14%,NaN
976,ASUS TUF Gaming F15 with 90WHr Battery Core i5...,Intel Core i5 Processor (11th Gen)16 GB DDR4 R...,"₹69,990","₹89,990",22%,NaN
977,Infinix Y1 Plus Neo (2023) Celeron Quad Core 1...,Intel Celeron Quad Core Processor (11th Gen)4 ...,"₹20,990","₹29,990",30%,NaN
978,DELL Core i5 11th Gen - (16 GB/512 GB SSD/Wind...,Intel Core i5 Processor (11th Gen)16 GB DDR4 R...,"₹65,500","₹91,000",28%,NaN
979,"ASUS Vivobook Pro 15 For Creator, Ryzen 5 Hexa...",AMD Ryzen 5 Hexa Core Processor8 GB DDR4 RAMWi...,"₹57,990","₹76,990",24%,NaN
980,Infinix ZEROBOOK 13 Intel Core i5 13th Gen - (...,Intel Core i5 Processor (13th Gen)16 GB LPDDR5...,"₹51,990","₹79,990",35%,NaN
981,HP Chromebook x360 Intel Celeron Dual Core - (...,Intel Celeron Dual Core Processor4 GB LPDDR4 R...,"₹17,990","₹32,295",44%,4.2
982,Lenovo Ryzen 3 Dual Core 3250U - (4 GB/1 TB HD...,AMD Ryzen 3 Dual Core Processor4 GB DDR4 RAM64...,"₹29,490","₹36,923",20%,3.6
983,Avita Liber Core i7 8th Gen - (8 GB/256 GB SSD...,Intel Core i7 Processor (8th Gen)8 GB DDR4 RAM...,"₹67,990","₹79,990",15%,NaN


In [9]:
# Guardamos nuestra información

flipk_data.to_csv('FlipK_Data.csv',index=False)

In [10]:
# Y la volvemos a cargar para comprobar que se ha guardado correctamente

ruta='FlipK_Data.csv'
data=pd.read_csv(ruta,sep=',')
data

,Product Name,Description,Price rupee,Original Price,Discount,Rating
0,Lenovo Slim 3 Chromebook Touch Intel Celeron D...,Intel Celeron Dual Core Processor4 GB LPDDR4 R...,"₹18,990","₹39,990",52%,NaN
1,ASUS Chromebook Touch Intel Celeron Dual Core ...,Intel Celeron Dual Core Processor4 GB LPDDR4 R...,"₹18,990","₹30,990",38%,NaN
2,ASUS Vivobook 15 Core i3 11th Gen - (8 GB/512 ...,Intel Core i3 Processor (11th Gen)8 GB DDR4 RA...,"₹34,990","₹49,990",30%,NaN
3,ASUS TUF Gaming A15 Ryzen 5 Hexa Core AMD R5-4...,AMD Ryzen 5 Hexa Core Processor8 GB DDR4 RAM64...,"₹52,990","₹75,990",30%,4.3
4,realme Book (Slim) Core i3 11th Gen - (8 GB/25...,Stylish & Portable Thin and Light Laptop14 inc...,"₹31,999","₹54,999",41%,NaN
...,...,...,...,...,...,...
979,"ASUS Vivobook Pro 15 For Creator, Ryzen 5 Hexa...",AMD Ryzen 5 Hexa Core Processor8 GB DDR4 RAMWi...,"₹57,990","₹76,990",24%,NaN
980,Infinix ZEROBOOK 13 Intel Core i5 13th Gen - (...,Intel Core i5 Processor (13th Gen)16 GB LPDDR5...,"₹51,990","₹79,990",35%,NaN
981,HP Chromebook x360 Intel Celeron Dual Core - (...,Intel Celeron Dual Core Processor4 GB LPDDR4 R...,"₹17,990","₹32,295",44%,4.2
982,Lenovo Ryzen 3 Dual Core 3250U - (4 GB/1 TB HD...,AMD Ryzen 3 Dual Core Processor4 GB DDR4 RAM64...,"₹29,490","₹36,923",20%,3.6


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Product Name    984 non-null    object 
 1   Description     984 non-null    object 
 2   Price rupee     984 non-null    object 
 3   Original Price  962 non-null    object 
 4   Discount        959 non-null    object 
 5   Rating          437 non-null    float64
dtypes: float64(1), object(5)
memory usage: 46.2+ KB
